# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [35]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [36]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [37]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [38]:
# Number of rows in csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


#### Creating a Cluster

In [39]:
# Connection to a Cassandra local instance  
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
 cluster = Cluster(['127.0.0.1'])
  # Session
 session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [40]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [41]:
# Set KEYSPACE sparkify
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)


In [42]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
## PRIMARY KEY: session_id is the partition key and item_insession an clustering column based in WHERE clause.

query = "CREATE TABLE IF NOT EXISTS event_song "
query = query + "(session_id INT, item_insession INT, artist_name TEXT, song_title TEXT, song_length DOUBLE, \
PRIMARY KEY (session_id, item_insession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [43]:
# csv file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## INSERT statements into the query variable
        query = "INSERT INTO event_song (session_id, item_insession, artist_name, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5]) ))

In [44]:
## SELECT statement to verify the data was entered into the table

query = "select artist_name, song_title, song_length from event_song WHERE session_id =338 and item_insession =4 "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.3073


In [45]:
#### Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) \
## for userid = 10, sessionid = 182
## PRIMARY KEY: userid is the partition key and sessionid and itemInSession the clustering columns based in WHERE clause.
## userid and sessionid columns was based in where clause. itemInSession column has been chosen becouse the sorting request
## and ensure unique values.

query = "CREATE TABLE IF NOT EXISTS play_list "
query = query + "(user_id INT,  session_id INT, item_insession INT, artist_name TEXT, song_title TEXT, user_firstname TEXT, \
user_lastname TEXT,  PRIMARY KEY ((user_id, session_id), item_insession))"

try:
    session.execute(query)
except Exception as e:
    print(e) 

In [46]:
# csv file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## INSERT statements into the query variable
        query = "INSERT INTO play_list (user_id, session_id, item_insession, artist_name, song_title, user_firstname, user_lastname)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4] ))

In [47]:
## SELECT statement to verify the data was entered into the table

query = "select artist_name, song_title, user_firstname, user_lastname FROM play_list WHERE user_id = 10 AND session_id = 182 "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.user_firstname, row.user_lastname)     

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [48]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
## PRIMARY KEY: song_title is the partition key and user_id is an clustering column. The column user_id was defined to ensure 
## unique values. 

query = "CREATE TABLE IF NOT EXISTS history_song "
query = query + "(song_title TEXT, user_id INT, user_firstname TEXT, user_lastname TEXT,  PRIMARY KEY (song_title, user_id))"

try:
    session.execute(query)
except Exception as e:
    print(e)             

In [49]:
# csv file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## INSERT statements into the query variable
        query = "INSERT INTO history_song (song_title, user_id, user_firstname, user_lastname)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [50]:
## SELECT statement to verify the data was entered into the table
query = "select user_firstname, user_lastname FROM history_song WHERE song_title = 'All Hands Against His Own' "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.user_firstname, row.user_lastname)    

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [51]:
## Drop the table before closing out the sessions
query = "drop table event_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table play_list"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table history_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)    
    

### Close the session and cluster connection¶

In [53]:
session.shutdown()
cluster.shutdown()